# Urban Heat Islands: India

## Import packages

In [ ]:
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, plot, utils

plt.style.use("seaborn-v0_8-notebook")

## Define Parameters

In [ ]:
# Region
area = [15, 75, 10, 80]

# Zoom
lon_slice = slice(77.4, 77.8)
lat_slice = slice(12.8, 13.2)

# Time Periods
satelite_years = {"spot": ["1999", "2000"], "proba": ["2014", "2015"]}

## Define Requests

In [ ]:
collection_id_albedo = "satellite-albedo"
common_albedo_request = {
    "format": "zip",
    "variable": "albb_bh",
    "sensor": "vgt",
    "product_version": "v2",
    "horizontal_resolution": "1km",
    "month": [f"{month:02d}" for month in range(1, 13)],
    "nominal_day": "10",
    "area": area,
}
albedo_requests = {
    satellite: common_albedo_request | {"satellite": satellite, "year": years}
    for satellite, years in satelite_years.items()
}

collection_id_temperature = "reanalysis-era5-land-monthly-means"
common_temperature_request = {
    "format": "grib",
    "product_type": "monthly_averaged_reanalysis",
    "variable": "2m_temperature",
    "time": "00:00",
    "month": [f"{month:02d}" for month in range(1, 13)],
    "area": area,
}
temperature_requests = {
    satellite: common_temperature_request | {"year": years}
    for satellite, years in satelite_years.items()
}

## Functions to cache

In [ ]:
def regionalised_func(ds, lon_slice, lat_slice, func):
    ds = utils.regionalise(ds, lon_slice=lon_slice, lat_slice=lat_slice)
    return func(ds)

## Download and transform albedo

In [ ]:
albedo_kwargs = {
    "chunks": {"year": 1, "month": 1},
    "drop_variables": ["crs"],
}

datasets = []
for satellite, request in albedo_requests.items():
    ds = download.download_and_transform(
        collection_id_albedo,
        request | {"area": area},
        transform_func=diagnostics.time_weighted_mean,
        transform_chunks=False,
        **albedo_kwargs,
    )
    datasets.append(ds.expand_dims(satellite=[satellite]))
ds_albedo_maps = xr.concat(datasets, "satellite")
ds_albedo_maps_zoom = utils.regionalise(
    ds_albedo_maps, lon_slice=lon_slice, lat_slice=lat_slice
)

datasets = []
for satellite, request in albedo_requests.items():
    ds = download.download_and_transform(
        collection_id_albedo,
        request | {"area": area},
        transform_func=diagnostics.seasonal_weighted_mean,
        transform_chunks=False,
        **albedo_kwargs,
    )
    datasets.append(ds.expand_dims(satellite=[satellite]))
ds_albedo_season_maps = xr.concat(datasets, "satellite")
ds_albedo_season_maps_zoom = utils.regionalise(
    ds_albedo_season_maps, lon_slice=lon_slice, lat_slice=lat_slice
)


albedo_timeseries_zoom = {}
for satellite, request in albedo_requests.items():
    albedo_timeseries_zoom[satellite] = download.download_and_transform(
        collection_id_albedo,
        request,
        transform_func=regionalised_func,
        transform_func_kwargs={
            "lon_slice": lon_slice,
            "lat_slice": lat_slice,
            "func": diagnostics.spatial_weighted_mean,
        },
        **albedo_kwargs,
    )

## Download and transform temperature

In [ ]:
temperature_kwargs = {
    "chunks": {"year": 1},
}

temperature_timeseries_zoom = {}
for satellite, request in temperature_requests.items():
    temperature_timeseries_zoom[satellite] = download.download_and_transform(
        collection_id_temperature,
        request,
        transform_func=regionalised_func,
        transform_func_kwargs={
            "lon_slice": lon_slice,
            "lat_slice": lat_slice,
            "func": diagnostics.spatial_weighted_mean,
        },
        **temperature_kwargs,
    )

## Plot albedo maps

In [ ]:
custom_cmap_s = mcolors.LinearSegmentedColormap.from_list(
    "CustomColorMap",
    ["blue", "skyblue", "cyan", "lightgreen", "yellow", "orange", "red", "maroon"],
)
for ds in (ds_albedo_maps, ds_albedo_maps_zoom):
    facet = plot.projected_map(ds["AL_BH_BB"], cmap=custom_cmap_s, row="satellite")
    plt.show()

## Plot albedo timeseries

In [ ]:
for satellite, ds in albedo_timeseries_zoom.items():
    ds["AL_BH_BB"].plot(marker="o", linestyle="-", color="b")
    plt.grid()
    plt.title(f"{satellite=}")
    plt.show()

## Define plotting function

In [ ]:
def compare_albedo_and_temperature(da_albedo, da_temperature, freq):
    # Resample
    da_albedo = da_albedo.resample(time=freq).mean()
    da_temperature = da_temperature.resample(forecast_reference_time=freq).mean()
    dt = da_temperature["forecast_reference_time"].diff("forecast_reference_time")

    # Convert units
    with xr.set_options(keep_attrs=True):
        da_temperature = da_temperature - 273.15
    da_temperature.attrs["units"] = "°C"

    fig, ax1 = plt.subplots()
    # Plot albedo
    color = "blue"
    da_albedo.plot(ax=ax1, color=color)
    ax1.set_ylabel(ax1.get_ylabel(), color=color)
    ax1.tick_params(axis="y", labelcolor=color)
    ax1.grid()

    # Plot temperature
    ax2 = ax1.twinx()
    color = "red"
    ax2.bar(
        da_temperature["forecast_reference_time"],
        da_temperature,
        width=dt[0] / 2,
        color=color,
        alpha=0.6,
    )
    ax2.set_ylabel(
        f"{da_temperature.attrs['long_name']} [{da_temperature.attrs['units']}]",
        color=color,
    )
    ax2.tick_params(axis="y", labelcolor=color)
    return fig, (ax1, ax2)

## Plot albedo vs temperature timeseries

In [ ]:
for freq in ("1MS", "QS-DEC"):
    for satellite, ds_albedo in albedo_timeseries_zoom.items():
        da_albedo = ds_albedo["AL_BH_BB"]
        da_temperature = temperature_timeseries_zoom[satellite]["t2m"]
        fig, (ax1, ax2) = compare_albedo_and_temperature(
            da_albedo, da_temperature, freq
        )
        fig.suptitle(f"{satellite=} {freq=}")
        plt.show()

## Plot albedo seasonality

In [ ]:
for ds in (ds_albedo_season_maps, ds_albedo_season_maps_zoom):
    facet = plot.projected_map(
        ds["AL_BH_BB"], cmap=custom_cmap_s, row="satellite", col="season"
    )
    plt.show()